In [1]:
#!/usr/bin/env python3
"""
state_attribute_counts_fixed.py
Improved production-ready script to compute question counts per state per attribute
from CSVs containing graph_path data.

Usage:
  python state_attribute_counts_fixed.py --input "/path/Full Dataset.csv" --outdir "/path/output"
"""
import argparse
import logging
import os
import re
import sys
from typing import List, Optional, Set

import pandas as pd

# ------------------------
# Logging config
# ------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# ------------------------
# Helper functions
# ------------------------
def safe_read_csv(path: str) -> Optional[pd.DataFrame]:
    """Read CSV with fallbacks for common encoding/engine issues."""
    try:
        df = pd.read_csv(path)
        logger.info("Loaded %d rows from %s", len(df), path)
        return df
    except Exception as e1:
        logger.warning("pd.read_csv failed: %s — trying engine='python' with utf-8-sig", e1)
        try:
            df = pd.read_csv(path, engine="python", encoding="utf-8-sig", on_bad_lines='skip')
            logger.info("Loaded %d rows (fallback) from %s", len(df), path)
            return df
        except Exception as e2:
            logger.error("Failed to read %s: %s", path, e2)
            return None


def find_graph_columns(df: pd.DataFrame) -> List[str]:
    """Return list of columns that look like graph_path or graph_path/0 etc."""
    cols = df.columns.tolist()
    candidates = [c for c in cols if re.search(r'graph|path', c, flags=re.I)]
    if any(c.lower() == "graph_path" for c in candidates):
        return ["graph_path"]
    return sorted(candidates)


def merge_graph_columns(df: pd.DataFrame, graph_cols: List[str]) -> pd.Series:
    """Merge multiple graph columns into a single combined_graph string per row."""
    if not graph_cols:
        return pd.Series([""] * len(df), index=df.index)
    if len(graph_cols) == 1:
        return df[graph_cols[0]].astype(str).fillna("")
    
    def join_row_parts(row):
        parts = []
        for c in graph_cols:
            v = row.get(c)
            if pd.isna(v):
                continue
            s = str(v).strip()
            if s:
                parts.append(s)
        return "|".join(parts)

    return df.apply(join_row_parts, axis=1)


def extract_state_from_graph(text: str) -> Optional[str]:
    """Try to extract the state token from combined graph string."""
    if not isinstance(text, str) or not text:
        return None
    m = re.search(r'state[:=]([A-Za-z0-9_ \-]+)', text, flags=re.I)
    if m:
        return m.group(1).strip().title()
    return None


def detect_attributes_from_graphs(series: pd.Series) -> Set[str]:
    """Scan combined_graph series and return set of attribute keys (left of ':')."""
    attrs = set()
    for text in series.dropna().astype(str):
        tokens = re.split(r'\||;', text)
        for t in tokens:
            t = t.strip()
            if not t or ':' not in t:
                continue
            key = t.split(':', 1)[0].strip().lower()
            if key and key != "state":
                attrs.add(key)
    return attrs


def make_attribute_flags(df: pd.DataFrame, combined_graph_col: str, attributes: List[str]) -> pd.DataFrame:
    """Create 0/1 columns in df for each attribute."""
    for att in attributes:
        pattern = rf'(?i)\b{re.escape(att)}:'
        df[att] = df[combined_graph_col].astype(str).str.contains(pattern, regex=True, na=False).astype(int)
    return df


# ------------------------
# Core processing
# ------------------------
def process_file(input_path: str, outdir: str, dedup_col: Optional[str] = "Corrected Question"):
    logger.info("Processing file: %s", input_path)
    df = safe_read_csv(input_path)
    if df is None:
        raise RuntimeError(f"Failed to load {input_path}")

    graph_cols = find_graph_columns(df)
    if not graph_cols:
        raise RuntimeError(f"No graph-like column found in {input_path}")

    logger.info("Found graph columns: %s", graph_cols)
    df["__combined_graph"] = merge_graph_columns(df, graph_cols)

    # Extract state
    df["__state_extracted"] = df["__combined_graph"].apply(extract_state_from_graph)
    state_col_candidates = [c for c in df.columns if c.lower().strip() == "state"]
    df["__state_fallback"] = df[state_col_candidates[0]].astype(str).str.strip().replace({"nan": ""}) if state_col_candidates else ""

    def pick_state(row):
        if row["__state_extracted"]:
            return row["__state_extracted"]
        if row["__state_fallback"]:
            return row["__state_fallback"].title()
        return "UNKNOWN"

    df["state_final"] = df.apply(pick_state, axis=1)

    # Detect attributes
    detected_attrs = detect_attributes_from_graphs(df["__combined_graph"])
    attributes = sorted(list(detected_attrs)) if detected_attrs else ["tourism", "history", "art", "festival", "cuisine", "personalities", "costume"]
    logger.info("Using attributes: %s", attributes)

    # Create flags
    df = make_attribute_flags(df, "__combined_graph", attributes)

    # Raw counts grouped by state
    group = df.groupby("state_final")[attributes].sum().reset_index().rename(columns={"state_final": "state"})
    group["Total_Questions"] = group[attributes].sum(axis=1).astype(int)

    # Save raw counts
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    raw_out = os.path.join(outdir, f"{base_name}_state_attribute_counts_raw.csv")
    group.to_csv(raw_out, index=False)
    logger.info("Saved raw state-attribute counts to %s", raw_out)

    # Unique counts if dedup_col exists
    unique_out = None
    if dedup_col in df.columns:
        df_unique = df.dropna(subset=[dedup_col]).drop_duplicates(subset=[dedup_col])
        group_unique = df_unique.groupby("state_final")[attributes].sum().reset_index().rename(columns={"state_final": "state"})
        group_unique["Total_Questions"] = group_unique[attributes].sum(axis=1).astype(int)
        unique_out = os.path.join(outdir, f"{base_name}_state_attribute_counts_unique.csv")
        group_unique.to_csv(unique_out, index=False)
        logger.info("Saved unique-question counts to %s", unique_out)
    else:
        logger.info("Deduplication column '%s' not found; skipping unique counts.", dedup_col)

    # Save percent-augmented file
    pct_df = group.copy()
    for att in attributes:
        pct_df[f"{att}_pct"] = (pct_df[att] / pct_df["Total_Questions"].replace({0: 1})) * 100
    pct_out = os.path.join(outdir, f"{base_name}_state_attribute_counts_with_percent.csv")
    pct_df.to_csv(pct_out, index=False)
    logger.info("Saved percent-augmented file to %s", pct_out)

    return {"raw": raw_out, "unique": unique_out, "percent": pct_out, "attributes": attributes}


# ------------------------
# CLI main
# ------------------------
def main(argv: Optional[List[str]] = None):
    parser = argparse.ArgumentParser(description="Compute question counts per state per attribute from CSVs with graph_path.")
    parser.add_argument("--input", "-i", nargs="+", required=True, help="Input CSV file(s).")
    parser.add_argument("--outdir", "-o", required=True, help="Output directory for CSV summary files.")
    parser.add_argument("--dedup-col", "-d", default="Corrected Question", help="Column to use for unique-question deduplication.")
    args = parser.parse_args(argv)

    outdir = args.outdir
    os.makedirs(outdir, exist_ok=True)

    results = {}
    for input_path in args.input:
        if not os.path.isfile(input_path):
            logger.error("Input file not found: %s", input_path)
            continue
        try:
            info = process_file(input_path, outdir, dedup_col=args.dedup_col)
            results[input_path] = info
        except Exception as e:
            logger.exception("Failed processing %s: %s", input_path, e)

    logger.info("DONE. Generated outputs for %d files.", len(results))
    if not results:
        logger.error("No outputs generated. Check input file paths and format.")
        sys.exit(2)


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --input INPUT [INPUT ...] --outdir OUTDIR
                             [--dedup-col DEDUP_COL]
ipykernel_launcher.py: error: the following arguments are required: --input/-i, --outdir/-o


SystemExit: 2

/Users/anjalisingh/Library/Python/3.13/lib/python/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# -------------------------------
# Question Count per State per Attribute (Combined Datasets)
# -------------------------------

import pandas as pd
import re
import os

# -------------------------------
# 1. CONFIG: Paths to datasets
# -------------------------------
dataset_final = "/Users/anjalisingh/Desktop/IITP/QUESTION_COUNT/Full Dataset - Final Dataset.csv"
dataset_3hop = "/Users/anjalisingh/Desktop/IITP/QUESTION_COUNT/Full Dataset - 3_hop_questions.csv"
output_dir = os.path.dirname(dataset_final)
output_path = os.path.join(output_dir, "Combined_State_Attribute_Question_Count.csv")

# -------------------------------
# 2. LOAD DATASETS
# -------------------------------
def load_csv(path):
    try:
        df = pd.read_csv(path)
        print(f"✅ Loaded {os.path.basename(path)}: {df.shape[0]} rows, {df.shape[1]} columns")
        return df
    except Exception as e:
        raise FileNotFoundError(f"❌ Error loading {path}: {e}")

df_final = load_csv(dataset_final)
df_3hop = load_csv(dataset_3hop)

# Combine both datasets
df = pd.concat([df_final, df_3hop], ignore_index=True)
df = df.drop_duplicates(subset=['question'])  # optional, if same questions exist
print(f"📦 Combined dataset: {df.shape[0]} rows, {df.shape[1]} columns")

# -------------------------------
# 3. MERGE ALL GRAPH / SUPPORTING COLUMNS
# -------------------------------
# Detect relevant columns
possible_cols = [c for c in df.columns if 'graph' in c.lower() or 'support' in c.lower()]
print(f"Detected potential graph/support columns: {possible_cols}")

# Combine into a single string per row
df['combined_paths'] = df[possible_cols].astype(str).agg('|'.join, axis=1)

# -------------------------------
# 4. EXTRACT STATE AND ATTRIBUTES
# -------------------------------
def extract_info(text):
    """
    Extract state and all attributes like tourism, history, art, festival, cuisine, costume, religion, sports, personalities, medicine
    """
    if pd.isna(text):
        return None, []

    state_match = re.search(r'state:([a-zA-Z_\s]+)', text.lower())
    state = state_match.group(1).strip() if state_match else None

    attrs = re.findall(r'(tourism|art|history|festival|cuisine|costume|religion|sports|personalities|medicine):', text.lower())
    return state, attrs

df[['state', 'attributes']] = df['combined_paths'].apply(lambda x: pd.Series(extract_info(x)))

# Drop rows without state
df = df.dropna(subset=['state'])

# -------------------------------
# 5. COUNT QUESTIONS PER STATE-ATTRIBUTE
# -------------------------------
rows = []
for _, row in df.iterrows():
    state = row['state']
    for attr in row['attributes']:
        rows.append({'state': state, 'attribute': attr})

count_df = pd.DataFrame(rows)

# Aggregate counts
pivot_df = count_df.pivot_table(index='state', columns='attribute', aggfunc='size', fill_value=0)

# Add total question count per state
# Count unique questions per state
unique_questions_per_state = df.groupby('state').size()
pivot_df['total_questions'] = pivot_df.index.map(unique_questions_per_state)


# Sort by total questions
pivot_df = pivot_df.sort_values(by='total_questions', ascending=False)

# Reset index for clean output
pivot_df = pivot_df.reset_index()

# -------------------------------
# 6. SAVE + DISPLAY
# -------------------------------
pivot_df.to_csv(output_path, index=False)
print(f"✅ Saved combined state-attribute question count to:\n{output_path}\n")

print("📊 SAMPLE OUTPUT:")
print(pivot_df.head(10).to_string(index=False))


✅ Loaded Full Dataset - Final Dataset.csv: 3284 rows, 9 columns
✅ Loaded Full Dataset - 3_hop_questions.csv: 155 rows, 11 columns
📦 Combined dataset: 3380 rows, 18 columns
Detected potential graph/support columns: ['supporting_facts', 'graph_path', 'supporting_facts/0', 'supporting_facts/1', 'graph_path/0', 'graph_path/1', 'graph_path/2', 'graph_path/3']
✅ Saved combined state-attribute question count to:
/Users/anjalisingh/Desktop/IITP/QUESTION_COUNT/Combined_State_Attribute_Question_Count.csv

📊 SAMPLE OUTPUT:
            state  art  costume  cuisine  festival  history  medicine  personalities  religion  sports  tourism  total_questions
        telangana   97       48       16         0      450         0            268        55      14      347              667
       tamil_nadu   39       58       55       121      115        32              5        69       0       40              267
        karnataka   19       13       75       146      109         0              9         0 